In [1]:
from tqdm.notebook import tqdm
from IPython.display import display, HTML

import numpy as np
import pandas as pd

import plotly.graph_objects as go

import datetime
from pathlib import Path

import intake

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from tensorflow.keras.regularizers import l2
import tensorflow as tf
from tensorflow.keras import backend as K

from tensorflow.keras.layers import ConvLSTM2D, Conv2D, Conv3D, Dense, TimeDistributed, MaxPooling2D, GlobalAveragePooling2D 
from tensorflow.keras.layers import Activation, Dropout, Flatten, BatchNormalization, Input, LSTM

2024-05-18 14:01:06.986255: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-18 14:01:07.870786: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
catalog = intake.open_catalog(Path('../catalog.yml'))
source = getattr(catalog, 'treesat_multi')
df = source.read()

In [3]:
selected_bands = [f'B{x}' for x in range(2, 9)] + ['B8A', 'B11', 'B12', 'TCI_R', 'TCI_G', 'TCI_B']

In [4]:
target = source.metadata['categories']['multi'] # multi / trinary
labels = df[target].to_numpy()
majority_labels = (labels==labels.max(axis=1, keepdims=True)).astype(float)

In [5]:
model_dir = Path('models').joinpath('seasons')
model_dir.mkdir(parents=True, exist_ok=True)

data_dir = Path('seasonal_median')

test_year = 2019
test_seasons = [6]

train_years = [2017, 2018, 2020]
all_seasons = [3, 6, 9, 12]

In [6]:
class KerasModelCreator:
    def normalise_X(self, X, p=1):
        upper = np.percentile(X, 100-p)
        lower = np.percentile(X, p)
    
        bounded_X = np.where(X > upper, np.median(X), X)
        bounded_X = np.where(X < lower, np.median(X), bounded_X)
        
        scaled_X = (bounded_X - lower)/(upper - lower)
        return scaled_X
        
    def split_and_normalise(self, y, X, random_state=42, test_size=0.1):
        """Split and max scale."""
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=random_state, shuffle=True)

        for i in range(X_train.shape[-1]):
            X_train[...,i] = self.normalise_X(X_train[...,i])
            X_test[...,i] = self.normalise_X(X_test[...,i])
        
        return X_train, X_test, y_train, y_test

    def build_model(self, y_train, input_shape, metrics):
        filter_base= 32
        dropout_base = 0.5
        
        m = tf.keras.Sequential()

        m.add(Input(input_shape[1:]))
        for kernel_size in [5]:
            m.add(Conv2D(
                filters=filter_base, kernel_size=kernel_size, 
                padding='same', activation='relu'
            ))
            m.add(BatchNormalization())
            m.add(Dropout(dropout_base))
        
        m.add(Flatten())

        # for filter_scale in [4, 2, 1]:
        #     m.add(Dense(filter_scale*filter_base, activation='relu'))
        
        m.add(Dense(
            y_train.shape[1], 
            activation='softmax', 
        ))

        loss = tf.keras.losses.CategoricalCrossentropy()
        opt = tf.keras.optimizers.Adam(learning_rate=0.001)
        m.compile(
            optimizer=opt,
            loss=loss,
            metrics=metrics
        )
        return m

        
    def run(self, y, X, model_path, epochs=10, overwrite=False):
        ''' 
        If not overwrite and there's an existing model, the model will 
        continue training if the given epoch is bigger than the previous,
        else just evaluate.
        Ensure train splits are the same across continuations / evaluations
        by not modifying the random_state in split_and_normalise.
        '''
        model_savepoint = model_path.parent.joinpath(model_path.stem)
        log_file = model_path.with_suffix('.log')

        if overwrite:
            for f in [model_path, log_file] + list(model_savepoint.glob('*')):
                f.unlink(missing_ok=True)

        X_train, X_test, y_train, y_test = self.split_and_normalise(y, X, random_state=42)

        f1_score = tf.keras.metrics.F1Score(average='weighted')

        default_metrics = ['accuracy']
        
        
        if model_path.is_file():
            model = tf.keras.models.load_model(model_path)
        else:
            model = self.build_model(y_train, X_train.shape, default_metrics + [f1_score])

        callbacks = [
            tf.keras.callbacks.BackupAndRestore(
                model_savepoint, save_freq='epoch', delete_checkpoint=False
            ),
            tf.keras.callbacks.CSVLogger(
                log_file, append=True
            )
        ]

        if log_file.is_file():
            df = pd.read_csv(log_file)[['epoch', 'loss'] + default_metrics + ['f1_score']]
            df['epoch'] += 1
            print('Previous training:')
            display(HTML(df.to_html(index=False)))
        
        model.fit(
            X_train, y_train, epochs=epochs, verbose=1, batch_size=10, callbacks=callbacks, 
            shuffle=True, validation_data=(X_test, y_test)
        )

        model.save(model_path)
        
        return model

In [7]:
for season in test_seasons:
    selected_data = []
    for year in train_years:
        file_key = f'processed_treesat_{year}{str(season).zfill(2)}.npy'
        filepath = data_dir.joinpath(file_key)
        with open(filepath, 'rb') as f:
            data = np.load(f)
        selected_data.append(data)
        
    features = np.concatenate(selected_data, axis=0)
    
    matching_labels = np.vstack([labels]*len(train_years))
    
    model_name = f'conv_seasons_{"_".join(map(str, test_seasons))}_years_{"_".join(map(str, train_years))}.keras'
    model_path = model_dir.joinpath(model_name)
    
    KerasModelCreator().run(
        matching_labels, features, model_path, epochs=10, overwrite=False)

Previous training:


epoch,loss,accuracy,f1_score
1,1.793797,0.460052,0.388678
2,1.687524,0.496214,0.428283
3,1.651200,0.509255,0.445335
4,1.628721,0.517776,0.456478
5,1.611693,0.523488,0.464697
6,1.602540,0.527259,0.469014
7,1.594274,0.530207,0.472991
8,1.587920,0.532412,0.475581
9,1.583843,0.534868,0.478575
10,1.580326,0.535007,0.479534


In [8]:
test_results = []
for season in test_seasons:
    file_key = f'processed_treesat_{test_year}{str(season).zfill(2)}.npy'
    filepath = data_dir.joinpath(file_key)
    with open(filepath, 'rb') as f:
        test_features = np.load(f)
    
    for i in range(test_features.shape[-1]):
        test_features[...,i] = KerasModelCreator().normalise_X(test_features[...,i])

    model_name = f'conv_seasons_{"_".join(map(str, test_seasons))}_years_{"_".join(map(str, train_years))}.keras'
    model_path = model_dir.joinpath(model_name)
    model = tf.keras.models.load_model(model_path)
    result = model.evaluate(test_features, majority_labels, verbose=0, return_dict=True)
    test_results.append(result)

In [9]:
test_results

[{'accuracy': 0.44236913323402405,
  'f1_score': 0.418423056602478,
  'loss': 1.8832262754440308}]